In [26]:
# make sure to use DEBIAS (3.9.19) kernel

In [3]:
from debiasm import DebiasMClassifier
import debiasm
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [151]:
np.random.seed(123)
n_samples = 96*5
n_batches = 5
n_features = 100

## the read count matrix
X = ( np.random.rand(n_samples, n_features) *1000).astype(int)

## the labels
y = np.random.rand(n_samples)>0.5

## the batches
batches = ( np.random.rand(n_samples) * n_batches ).astype(int)

In [4]:
type(y)

numpy.ndarray

In [5]:
batches

array([4, 4, 3, 3, 1, 2, 0, 1, 1, 4, 0, 2, 1, 4, 1, 1, 0, 4, 4, 4, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 3, 3, 0, 0, 2, 4, 2, 1, 4, 4, 4, 1, 1, 4, 2,
       1, 3, 4, 2, 4, 3, 4, 4, 2, 3, 1, 2, 2, 1, 1, 4, 3, 0, 3, 3, 0, 1,
       4, 0, 3, 3, 3, 2, 4, 1, 1, 0, 0, 3, 0, 4, 3, 1, 3, 2, 1, 3, 2, 4,
       3, 0, 2, 2, 1, 4, 4, 0, 4, 2, 1, 4, 1, 2, 2, 0, 2, 3, 3, 3, 1, 2,
       1, 2, 1, 2, 1, 3, 4, 4, 4, 0, 4, 4, 1, 4, 2, 2, 0, 3, 1, 0, 0, 3,
       3, 2, 4, 4, 3, 4, 4, 1, 3, 4, 2, 4, 4, 0, 4, 4, 2, 2, 2, 0, 2, 1,
       3, 4, 1, 4, 4, 1, 4, 1, 0, 2, 2, 3, 0, 4, 4, 4, 4, 4, 0, 0, 1, 0,
       4, 0, 1, 2, 1, 2, 2, 1, 3, 4, 3, 3, 4, 0, 3, 4, 0, 1, 3, 3, 0, 3,
       4, 4, 0, 3, 1, 3, 4, 3, 2, 2, 4, 0, 0, 4, 4, 4, 2, 4, 1, 3, 4, 0,
       3, 0, 0, 4, 3, 2, 2, 4, 4, 0, 2, 3, 1, 4, 0, 0, 1, 1, 1, 0, 4, 2,
       2, 4, 4, 1, 4, 2, 0, 4, 2, 2, 1, 3, 1, 3, 2, 4, 1, 2, 1, 0, 4, 1,
       3, 0, 2, 0, 2, 2, 2, 1, 1, 4, 1, 0, 4, 1, 0, 1, 3, 2, 0, 4, 0, 0,
       3, 0, 4, 1, 1, 3, 0, 3, 2, 3, 4, 4, 1, 3, 2,

In [6]:
## we assume the batches are numbered ints starting at '0',
## and they are in the first column of the input X matrices

X_with_batch = np.hstack((batches[:, np.newaxis], X))
X_with_batch[:10, :10]

array([[  4, 696, 286, 226, 551, 719, 423, 980, 684, 480],
       [  4, 513, 666, 105, 130, 321, 661, 846, 553, 854],
       [  3, 542,  66, 653, 996, 769, 573, 102, 699, 661],
       [  3,  16, 721,   7,  84, 225, 875, 363, 539, 568],
       [  1, 456, 279, 932, 314, 909,  43, 707, 483, 444],
       [  2, 212, 392, 851, 127, 893, 496, 426, 305, 916],
       [  0, 701, 707, 959, 876, 468, 625, 457, 222, 376],
       [  1, 945, 290, 727,  15, 879,  63, 733, 994, 501],
       [  1, 387, 555,  14, 847, 921, 550, 268, 990, 383],
       [  4, 896, 223, 268, 194, 967, 112, 722, 932, 668]])

In [7]:
## set the valdiation batch to '4'
val_inds = batches==4
X_train, X_val = X_with_batch[~val_inds], X_with_batch[val_inds]
y_train, y_val = y[~val_inds], y[val_inds]

In [8]:
dmc = DebiasMClassifier(x_val=X_val) ## give it the held-out inputs to account for
                                    ## those domains shifts while training

dmc.fit(X_train, y_train)
print('finished training!')

c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass 

finished training!


In [9]:
X_debiassed = dmc.transform(X_with_batch)
X_debiassed[:5, :5]

array([[0.01456792, 0.00533655, 0.00493307, 0.01202877, 0.01374545],
       [0.01067739, 0.01235743, 0.00227907, 0.0028221 , 0.00610232],
       [0.01087459, 0.00140587, 0.01448398, 0.02110646, 0.01628153],
       [0.00031825, 0.01522535, 0.00015392, 0.00176468, 0.0047226 ],
       [0.00967647, 0.00629353, 0.01726234, 0.00602933, 0.01834572]],
      dtype=float32)

## My own Data ##

### DEBIAS-M Leaky

In [9]:
np.random.seed(123)
log_DEBIAS = pd.read_csv("./csv_files/log_DEBIAS-M.csv") # read in data
data = log_DEBIAS.iloc[:,3:len(log_DEBIAS)] # select the case, Study ID (0-14), and data columns

data.head()

,case,ID,X.Anaerorhabdus..furcosa.group,X.Aquaspirillum..polymorphum.group,X.Bacteroides..pectinophilus.group,X.Clostridium..innocuum.group,X.Eubacterium..brachy.group,X.Eubacterium..eligens.group,X.Eubacterium..fissicatena.group,X.Eubacterium..hallii.group,...,Weissella,Wenxinia,Williamsia,Xanthobacter,Xanthomonas,XBB1006,Yersinia,Yokenella,Z20,Zymomonas
0,True,0,0.0,0.0,0.000000,0.0,0.0,2.736923,0.0,2.123979,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,True,0,0.0,0.0,0.000000,0.0,0.0,2.411039,0.0,1.579308,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,True,0,0.0,0.0,0.000000,0.0,0.0,3.349333,0.0,0.904502,...,1.408091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,True,0,0.0,0.0,1.555367,0.0,0.0,2.244556,0.0,1.358441,...,0.729595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,True,0,0.0,0.0,0.000000,0.0,0.0,2.398860,0.0,1.010379,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
## the read count matrix and "batches" a.k.a sample_IDs
X = data.iloc[:,2:len(data)]
X = X.to_numpy()

## select the labels
y =  data["case"].to_numpy() # the 0s and 1s... make sure true (case/1) false(control/0)

# batches
# batches = data.iloc[:,1:2]
# batches = batches.to_numpy()

# batches
batches = data.iloc[:, 1]
batches  = batches.to_numpy().astype(int)
batches


array([0, 0, 0, ..., 2, 2, 2])

In [12]:
## we assume the batches are numbered ints starting at '0',
## and they are in the first column of the input X matrices

# run this one without editing
# X_with_batch = np.hstack((batches[:, np.newaxis], X))
X_with_batch = np.hstack((batches[:, np.newaxis], X))
X_with_batch[:10, :10]
X_with_batch


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [2., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.]])

Run DEBIAS-M 15 times with 15 validation batches 
- will return 15 DEBIASED datasets and 15 weights
- RF on the 15 data sets with the testing on the val_inds 


In [13]:

## set the valdiation batch to '4'
# this means the training will be done without the fourth batch
# run this one without editing
val_inds = batches==4 
X_train, X_val = X_with_batch[~val_inds], X_with_batch[val_inds] 
y_train, y_val = y[~val_inds], y[val_inds]

In [157]:
print((X_train))
print("\n",X_val)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [2. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]]

 [[4. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 ...
 [4. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]]


In [14]:
# run this one without editing
dmc = DebiasMClassifier(x_val=X_val) ## give it the held-out inputs to account for
                                    ## those domains shifts while training



In [15]:
dmc.fit(X_train, y_train)
print('finished training!')

c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
c:\Users\brean\miniconda3\envs\DEBIAS\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass 

finished training!


In [142]:
roc_auc_score(y_val, dmc.predict_proba(X_val)[:, 1]) 

0.6884969037342841

In [16]:
# run this one without editing

X_debiassed = dmc.transform(X_with_batch)
X_debiassed[:5, :5]

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.00748433, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [144]:
weights = dmc.model.batch_weights
print(weights)
np_weights = weights.detach().numpy()
np_weights.shape

# weights.to_csv("./csv_files/debias_weights.csv")

Parameter containing:
tensor([[ 0.0000,  2.7649, -1.2656,  ...,  0.0000, -0.2166,  0.0000],
        [ 0.0000,  0.0000,  1.6813,  ...,  0.0000, -2.7007,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000, -1.0686,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.6207,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0253,  ...,  0.0000,  0.0000,  0.0000]],
       requires_grad=True)


(15, 921)

In [17]:
cols = list(data.columns[2:len(data)])

debiased_df = pd.DataFrame(data = X_debiassed,
                           columns = cols)

first_cols = log_DEBIAS.iloc[:, 0:3]

frames = [first_cols, debiased_df]

merged = pd.concat(frames, axis=1)

merged.to_csv("./csv_files/debiased_lognorm.csv", index = False)

merged


,sample_name,Phenotype,Study_ID,X.Anaerorhabdus..furcosa.group,X.Aquaspirillum..polymorphum.group,X.Bacteroides..pectinophilus.group,X.Clostridium..innocuum.group,X.Eubacterium..brachy.group,X.Eubacterium..eligens.group,X.Eubacterium..fissicatena.group,...,Weissella,Wenxinia,Williamsia,Xanthobacter,Xanthomonas,XBB1006,Yersinia,Yokenella,Z20,Zymomonas
0,10959.GBR00561.2011,1,10959,0.0,0.0,0.000000,0.000000,0.000000,0.016895,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10959.GBR00567.2011,1,10959,0.0,0.0,0.000000,0.000000,0.000000,0.012346,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10959.GBR00568.2011,1,10959,0.0,0.0,0.000000,0.000000,0.000000,0.022071,0.00000,...,0.000467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10959.GBR00571.2011,1,10959,0.0,0.0,0.007484,0.000000,0.000000,0.015359,0.00000,...,0.000251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10959.GBR00623.2011,1,10959,0.0,0.0,0.000000,0.000000,0.000000,0.012071,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3597,15006.9C1,1,15006,0.0,0.0,0.000000,0.026794,0.000000,0.008587,0.00432,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3598,15006.9F1,0,15006,0.0,0.0,0.000000,0.000000,0.002645,0.005697,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3599,15006.9M1,0,15006,0.0,0.0,0.000000,0.000000,0.000000,0.007866,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3600,15006.9SK1,0,15006,0.0,0.0,0.000000,0.000000,0.000000,0.032703,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
# Study_ID = pd.DataFrame(data = log_DEBIAS["Study_ID"].unique(),
#                         columns=["Study_ID"])
# weights_df = pd.DataFrame(data = np_weights,
#                           columns = cols)

# frames = [Study_ID, weights_df]

# weights_df = pd.concat(frames, axis = 1)
# weights_df["Study_ID"] = weights_df["Study_ID"].astype("category")
# weights_df.to_csv("./csv_files/debias_weights.csv")

# weights_df


,Study_ID,X.Anaerorhabdus..furcosa.group,X.Aquaspirillum..polymorphum.group,X.Bacteroides..pectinophilus.group,X.Clostridium..innocuum.group,X.Eubacterium..brachy.group,X.Eubacterium..eligens.group,X.Eubacterium..fissicatena.group,X.Eubacterium..hallii.group,X.Eubacterium..nodatum.group,...,Weissella,Wenxinia,Williamsia,Xanthobacter,Xanthomonas,XBB1006,Yersinia,Yokenella,Z20,Zymomonas
0,10959,0.000000,2.764934,-1.265646,1.358705,1.218808,-0.808572,0.654470,-0.208837,1.617939,...,-5.021760,0.000000,0.000000,2.509073,0.000000,3.400071,0.000000,0.00000,-0.216586,0.00000
1,11129,0.000000,0.000000,1.681281,3.984919,-2.203013,0.477271,0.370220,-0.047013,0.993556,...,-1.400982,-1.206997,1.556240,0.000000,-2.591092,-1.648795,3.545854,0.00000,-2.700722,0.00000
2,11210,0.000000,0.000000,0.000000,1.958127,-1.051986,-0.255628,-0.648740,-0.783837,-0.337025,...,2.550679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
3,11635,-1.896914,0.000000,1.630689,-0.035764,-2.957492,-0.486201,-0.207958,-1.083358,1.017479,...,0.454202,0.000000,0.000000,0.000000,0.000000,0.000000,2.744119,0.00000,-0.994668,2.59433
4,11710,2.515771,0.000000,0.627875,1.025706,-0.323922,0.749434,1.387799,0.084966,-0.356977,...,1.530811,0.000000,0.829007,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
5,11993,0.000000,0.000000,0.000000,-1.210472,0.522875,-0.793887,-1.950767,0.882423,-1.008212,...,2.598222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
6,13010,0.000000,0.000000,-4.088838,0.059340,-2.129273,-0.646098,0.129568,-0.915050,-2.110905,...,2.805486,0.000000,0.000000,0.000000,0.000000,4.412981,0.000000,0.00000,-0.686798,0.00000
7,13187,0.000000,0.000000,1.531150,-1.714435,1.905474,-1.299735,0.667357,1.406692,3.477773,...,1.381916,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
8,13338,-1.212751,0.000000,0.126770,-1.404443,-2.850213,-0.077395,-0.803737,-1.214628,-2.578348,...,-1.263471,0.000000,0.865218,0.000000,0.000000,0.000000,1.070979,0.00000,0.000000,0.00000
9,13631,0.000000,0.000000,0.840419,2.346562,1.398412,-0.257476,1.606096,-0.654431,1.490758,...,-0.836599,0.000000,0.000000,0.000000,0.000000,0.000000,-1.191989,3.23868,-4.215164,0.00000


In [198]:
Study_ID = log_DEBIAS["Study_ID"].unique()

weights_df = pd.DataFrame(data = np_weights,
                          columns = cols,
                          index = Study_ID)

weights_df = weights_df.transpose()
weights_df.to_csv("./csv_files/debias_weights.csv")

weights_df



,10959,11129,11210,11635,11710,11993,13010,13187,13338,13631,13652,13695,14130,14669,15006
X.Anaerorhabdus..furcosa.group,0.000000,0.000000,0.000000,-1.896914,2.515771,0.000000,0.000000,0.000000,-1.212751,0.000000,0.000000,0.000000,0.000000,0.620668,0.000000
X.Aquaspirillum..polymorphum.group,2.764934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
X.Bacteroides..pectinophilus.group,-1.265646,1.681281,0.000000,1.630689,0.627875,0.000000,-4.088838,1.531150,0.126770,0.840419,0.000000,-1.147599,-1.068602,0.000000,-0.025347
X.Clostridium..innocuum.group,1.358705,3.984919,1.958127,-0.035764,1.025706,-1.210472,0.059340,-1.714435,-1.404443,2.346562,-0.432224,-0.060109,2.649692,-0.200680,0.648599
X.Eubacterium..brachy.group,1.218808,-2.203013,-1.051986,-2.957492,-0.323922,0.522875,-2.129273,1.905474,-2.850213,1.398412,-0.674228,-1.231972,4.332458,-1.775725,0.283822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XBB1006,3.400071,-1.648795,0.000000,0.000000,0.000000,0.000000,4.412981,0.000000,0.000000,0.000000,0.000000,-1.391053,-0.415355,0.000000,0.000000
Yersinia,0.000000,3.545854,0.000000,2.744119,0.000000,0.000000,0.000000,0.000000,1.070979,-1.191989,2.622039,0.000000,0.000000,0.000000,0.000000
Yokenella,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.238680,0.000000,0.000000,0.000000,0.000000,0.000000
Z20,-0.216586,-2.700722,0.000000,-0.994668,0.000000,0.000000,-0.686798,0.000000,0.000000,-4.215164,0.000000,-3.574217,0.000000,0.000000,0.000000


In [18]:
i = 10
id_dict = {}
Study_ID = log_DEBIAS["Study_ID"].unique()
id_array = log_DEBIAS["ID"].unique()
for id in range(0,len(Study_ID)):
    id_dict[id_array[id]] = Study_ID[id]

type(id_dict)


In [114]:
debiased_df

,X.Anaerorhabdus..furcosa.group,X.Aquaspirillum..polymorphum.group,X.Bacteroides..pectinophilus.group,X.Clostridium..innocuum.group,X.Eubacterium..brachy.group,X.Eubacterium..eligens.group,X.Eubacterium..fissicatena.group,X.Eubacterium..hallii.group,X.Eubacterium..nodatum.group,X.Eubacterium..oxidoreducens.group,...,Wohlfahrtiimonas,Xanthobacter,Xanthomonas,XBB1006,Xylophilus,Yersinia,Yokenella,Z20,Zavarzinella,Zymomonas
0,0.0,0.0,0.000000,0.000000,0.000000,0.016373,0.000000,0.010655,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.000000,0.012030,0.000000,0.006608,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,0.022508,0.000000,0.005097,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.011283,0.000000,0.000000,0.015702,0.000000,0.007969,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.012793,0.000000,0.004519,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3597,0.0,0.0,0.000000,0.021537,0.000000,0.008962,0.004429,0.012209,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3598,0.0,0.0,0.000000,0.000000,0.002167,0.005647,0.000000,0.005916,0.004555,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3599,0.0,0.0,0.000000,0.000000,0.000000,0.007248,0.000000,0.002484,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3600,0.0,0.0,0.000000,0.000000,0.000000,0.032289,0.000000,0.003236,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
log_DEBIAS

,sample_name,Phenotype,Study_ID,case,ID,X.Anaerorhabdus..furcosa.group,X.Aquaspirillum..polymorphum.group,X.Bacteroides..pectinophilus.group,X.Clostridium..innocuum.group,X.Eubacterium..brachy.group,...,Wohlfahrtiimonas,Xanthobacter,Xanthomonas,XBB1006,Xylophilus,Yersinia,Yokenella,Z20,Zavarzinella,Zymomonas
0,10959.GBR00561.2011,1,10959,True,0,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
1,10959.GBR00567.2011,1,10959,True,0,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
2,10959.GBR00568.2011,1,10959,True,0,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3,10959.GBR00571.2011,1,10959,True,0,0.0,0.0,1.555367,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
4,10959.GBR00623.2011,1,10959,True,0,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3597,15006.9C1,1,15006,True,2,0.0,0.0,0.000000,0.487406,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3598,15006.9F1,0,15006,False,2,0.0,0.0,0.000000,0.000000,0.669462,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3599,15006.9M1,0,15006,False,2,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3600,15006.9SK1,0,15006,False,2,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
